# Exploring Crono MA-XRF data with the `cronomaxrf` package 

Recently our lab - the Rijkserfgoedlaboratorium in Amsterdam - acquired a new Bruker Crono X-Ray Fluorescence spectral image (MACRO-XRF) scanner. This instrument saves its raw data into a specific type of HDF5 file format. Currently these files can only be read and converted by commercially licensed Crono software. This situation restricts access to the data to a limited number of researchers who can run the software. Furthermore, it is likely that we will loose the ability to read the data at some point in the future.

In order to make optimal use of the Crono data in collaborative research projects now and to make sure that we can still read the data in the future we need to: 1) understand the Crono HDF5 file format, and 2) develop open software tools available to all researchers for reading and converting these data files. To support open heritage science cronomaxrf package is currently being developed under the GPLv3 open source software license.

For installation instructions, documentation and code see: [https://fligt.github.io/cronomaxrf/](https://fligt.github.io/cronomaxrf/).

## STEP 1 - Import the package 

Import the package in the standard way. This requires that it has been 'pip installed'. If you are running this notebook on binder, we already took care of this. 

In [ ]:
import cronomaxrf as crx

## STEP 2 - Download the demo data

First create a `downloads` sub directory.   

In [ ]:
crx.make_downloads_dir()

Then download the demo data. 

In [ ]:
crx.download()

## STEP 3 - Inspect the contents of the Crono HDF5 file 

An overview of all the datasets and (part of) their contents can be printed using the `report()` function.

In [ ]:
crono_filename = '14200215102021-blindTest2AgedDetail.HDF5'   # 50 Mb 
crx.report(crono_filename)

## STEP 4 - Plot the problem 



Now that we have a rough idea about the contents of the different datasets, we are ready to visualize the spectral data. As we shall see there is a problem with the motor positions that we need to resolve. In order to make clear what the issue is, we need to plot the (x,y) motor positions and the selected spectra. Information about the motor positions and selected spectra is stored in datasets **17** and **28**. 

In [ ]:
crx.plot_the_problem(crono_filename)

## STEP 5 - Converting the wobbly data into a regular grid 

Mm, the xy positions of the sensor are a bit wobbly. We observe 85 rows of selected (y) positions. In each row we count 37 selected spectra (red dots). In order create a regular spectral image cube, the Crono software throws away quite some data (all blue dots). It seems that one could do a better job by interpolating. But we still need to develop code for that. Interested? 

For now let's follow the crude but effective strategy of the Crono software for exporting to a regular grid (**gridify**) by selecting the red dot spectra and save as regular hdf5 file so that Ana can process the data with her own software. 

In [ ]:
crx.gridify(crono_filename)

## STEP 6 Our first element map 

The exported (gridified) HDF5 file contains the selected spectral data cube that we need for further processing. Typically one now proceeds to obtain element maps with special software. Instead we will see how this is done manually. As an example we can compute the iron map from the Fe_Ka band. 

The new HDF5 file contains only two datasets: a one-dimensional array of 4096 energies, and a three dimensional cube of $85\times37\times4096$ spectral intensities. 

In [ ]:
gridified_filename = '14200215102021-blindTest2AgedDetail_GRIDIFIED.HDF5'
datasets = crx.read_datasets(gridified_filename)

We can make a map for iron by integrating over the 6.1-6.7 keV energy band as follows.

In [ ]:
# Fe_Ka band 
energies = datasets[0][...] # shape (4096,) 
cube = datasets[1][...] # shape (85, 37, 4096)
is_iron_Ka_band = (energies > 6.1) * (energies < 6.7) 
n_channels = np.sum(is_iron_Ka_band) # number of channels in the Fe_Ka band 

# integrate FeKa band 
FeKa_slice = cube[:,:,is_iron_Ka_band]
FeKa_map = FeKa_slice.sum(axis=2) / n_channels 

And create a plot. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# make plot 
fig, ax = plt.subplots(figsize=[14, 10])
ax.imshow(FeKa_map, origin='lower')
ax.set_title('Fe_Ka map');

## STEP 7 - Comparison with a photo 

And here is the side by side comparison with area of Blind-Test-Ink-8 (aged) in our paper sample. 

In [ ]:
img_vis = plt.imread('Ink-08-aged-VIS.png')[100:1200, 50:1300] # roughly the scanning area
img_vis = img_vis.transpose([1, 0, 2])[:,::-1] # rotate 90 deg clockwise 

h, w, _ = img_vis.shape

fig, [ax, ax1] = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=[14, 6])
ax.imshow(FeKa_map, extent=[0, w, h, 0], origin='lower')
ax1.imshow(img_vis)
ax.set_title('Fe_Ka (deformed)')
ax1.set_title('Blind-Test-Ink-8 (aged)');

**That is it!** We no longer depend on the Crono software for exporting the spectral data in a regular hdf5 file. 